In [ ]:
import numpy as np
from glob import glob
import pandas as pd
from matplotlib import pyplot as plt
import cv2

In [ ]:
df_matches = pd.read_csv('data/scene_matches_simple_gen.csv')

In [ ]:
vid_dir = './data/'

In [ ]:
scene_metadata_path = "data/representations/scene_metadata.csv"
try:
    # detect_scenes must have been run with config.save_files == True for this to work
    scene_metadata = pd.read_csv(scene_metadata_path)
    scene_durations = []
    for _, row in scene_metadata.iterrows():
        scene_durations += [[row['video_filename'], row['scene_duration_seconds']]]
except Exception:
    print("Error loading scene metadata, file '%s' not found" % scene_metadata_path)

In [ ]:
for index, row in df_matches.iterrows():
    if np.random.rand() < 0.9:
        continue
    print('===============')
    filename_a = str(row[4])
    vid_a_path = vid_dir + filename_a
    filename_b = str(row[5])
    vid_b_path = vid_dir + filename_b
    scene_ind_a = int(row[6])
    scene_ind_b = int(row[7])
    print('Video a: ' + vid_a_path)
    print('Scene a: %d' % scene_ind_a)
    print('Video b: ' + vid_b_path)
    print('Scene b: %d' % scene_ind_b)
    print('Distance: ' + str(row[3]))
    
    scenes_dur_a = None
    for filename, durs in scene_durations:
        if filename in filename_a:
            scenes_dur_a = [int(d) for d in durs.strip('][').split(', ')]
            break
    if scenes_dur_a is None:
        raise Exception("Error: no scene metadata available for file '%s'" % filename_a)
        
    cap = cv2.VideoCapture(vid_a_path)
    frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS)+.5)
    print('FPS a: %d' % fps)
        
    pre_scene_dur_a = sum(scenes_dur_a[:scene_ind_a]) + scenes_dur_a[scene_ind_a]*0.5
    pre_scene_frames_a = int(pre_scene_dur_a * fps)
    
    cap.set(cv2.CAP_PROP_POS_MSEC, pre_scene_dur_a)
    #for i in range(pre_scene_frames_a+1):
    _, buf_a = cap.read()
    
    cap.release()
    
    scenes_dur_b = None
    for filename, durs in scene_durations:
        if filename in filename_b:
            scenes_dur_b = [int(d) for d in durs.strip('][').split(', ')]
            break
    if scenes_dur_b is None:
        raise Exception("Error: no scene metadata available for file '%s'" % filename_b)
        
    cap = cv2.VideoCapture(vid_b_path)
    frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS)+.5)
    print('FPS b: %d' % fps)
        
    pre_scene_dur_b = sum(scenes_dur_b[:scene_ind_b]) + scenes_dur_b[scene_ind_b]*0.5
    pre_scene_frames_b = int(pre_scene_dur_b * fps)
    
    cap.set(cv2.CAP_PROP_POS_MSEC, pre_scene_dur_b)
    #for i in range(pre_scene_frames_b+1):
    _, buf_b = cap.read()
    
    cap.release()
    
    print('Scene a duration: %d' % scenes_dur_a[scene_ind_a])
    print('Scene b duration: %d' % scenes_dur_b[scene_ind_b])
    
    plt.subplot(1, 2, 1)
    plt.imshow(buf_a)
    plt.subplot(1, 2, 2)
    plt.imshow(buf_b)
    plt.show()

